In [4]:
import pandas as pd
import numpy as np
import nltk
import json
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
#nltk.download('all', force=True)
#nltk.data.path.append("/Users/laurenceliao/nltk_data")

In [5]:
tech_data = pd.read_csv("tech_jobs_trainingset.csv")
tech_training = pd.DataFrame(tech_data[['jobtitle', 'jobdescription', 'skills', 'tech', 'tag']])

tech_training = tech_training.dropna()
tech_training = tech_training.reindex()

In [6]:
# Data Preprocessing
tech_training = tech_training.map(lambda x: x.lower() if isinstance(x, str) else x)
tech_training

old_word = r'\bit\b'

# This is to avoid the stopwords function from removing It since it reads it as "it"
tech_training['tag'] = tech_training['tag'].str.replace(old_word, 'information_technology', regex=True)
tech_training['tech'] = tech_training['tech'].str.replace(r'\bno\b', 'unrelated', regex=True)
tech_training['tech'] = tech_training['tech'].str.replace(r'\byes\b', 'tech', regex=True)

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Removes special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-Z\s]'  
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# remove whitespace
def remove_whitespace(text):
    return " ".join(text.split())


# remove keyfault stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

# Stemming 

stemmer = PorterStemmer()

def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    return stems

# Lemmatization
lemmatizer = WordNetLemmatizer

def lemma_words(text):
    word_tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_words
    

tech_training = tech_training.map(lambda x: remove_special_characters(x) if isinstance(x, str) else x)
tech_training = tech_training.map(lambda x: remove_punctuation (x) if isinstance(x, str) else x)
tech_training = tech_training.map(lambda x: remove_whitespace(x) if isinstance(x, str) else x)
tech_training = tech_training.map(lambda x: remove_stopwords(x) if isinstance(x, str) else x)
tech_training = tech_training.map(lambda x: stem_words(x) if isinstance(x, str) else x)
tech_training = tech_training.map(lambda x: lemma_words(x) if isinstance(x, str) else x)

In [8]:
from sklearn.preprocessing import LabelEncoder
# Combining both job title and job description
#tech_training['combined'] = tech_training['jobtitle'] + tech_training['jobdescription']
tech_training['combined'] = tech_training['jobtitle'] + tech_training['jobdescription'] + tech_training['skills'] 
# im not sure if this is right
X = tech_training['combined']




y = tech_training['tag'].apply(lambda x: x[0])


# Assuming 'y' contains your categorical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Train Test Split (80% training data, 20% testing data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Convert list of lists into list of strings
X_train_strings = [' '.join(words) for words in X_train]
X_test_strings = [' '.join(words) for words in X_test]

# Instantiate TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform training data
X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train_strings)
X_train_Tfidf_df = pd.DataFrame(X_train_Tfidf_df.toarray())

# Transform test data
X_test_Tfidf_df = tfidf_vectorizer.transform(X_test_strings)
X_test_Tfidf_df = pd.DataFrame(X_test_Tfidf_df.toarray())


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
# Testing Stratified K Fold instead of K test split
tech_training['combined'] = tech_training['jobtitle'] + tech_training['skills'] + tech_training['jobdescription'] 
X = tech_training['combined']


y = tech_training['tag'].apply(lambda x: x[0])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


X = [' '.join(inner_list) for inner_list in X]

accuracies = []
skf = StratifiedKFold(n_splits=9)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
    
    y_train, y_test = y[train_index], y[test_index]
    # Initialize 
    tfidf_vectorizer = TfidfVectorizer()

    # Fit transform the vectorizer

    X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train).toarray()


    X_test_Tfidf_df = tfidf_vectorizer.transform(X_test).toarray()


    #tfidf_vectorizer.get_feature_names_out()
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust parameters like n_estimators

# Train the classifier
    rf_classifier.fit(X_train_Tfidf_df, y_train)

# Predict on the test data
    y_pred_rf = rf_classifier.predict(X_test_Tfidf_df)

# Evaluate the classifier
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    accuracies.append(accuracy_rf)
    #print("Random Forest Accuracy:", accuracy_rf)
print(np.mean(accuracies))


C:\Users\frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=9.
  warnings.warn(


0.7108764008707572


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Testing Stratified K Fold instead of K test split
tech_training['combined'] = tech_training['jobtitle'] + tech_training['skills'] + tech_training['jobdescription'] 
X = tech_training['combined']


y = tech_training['tag'].apply(lambda x: x[0])


X = [' '.join(inner_list) for inner_list in X]

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
    y_train, y_test =  y.iloc[train_index], y.iloc[test_index]
    # Initialize 
    tfidf_vectorizer = TfidfVectorizer()

    # Fit transform the vectorizer

    X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train).toarray()


    X_test_Tfidf_df = tfidf_vectorizer.transform(X_test).toarray()


    #tfidf_vectorizer.get_feature_names_out()
    X_train_Tfidf_df

    model = CategoricalNB()

    model.fit(X_train_Tfidf_df, y_train)

    y_pred = model.predict(X_test_Tfidf_df)

    accuracy = accuracy_score(y_test, y_pred)

    print("Accuracy: ", accuracy)

    if y_pred.ndim == 1:
        y_pred = pd.Series(y_pred, index=y_test )
    else:
        y_pred = pd.DataFrame(y_pred, index=y_test.index)


C:\Users\frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Accuracy:  0.45422535211267606
Accuracy:  0.4507042253521127
Accuracy:  0.4558303886925795
Accuracy:  0.4558303886925795
Accuracy:  0.4558303886925795


In [18]:
model = CategoricalNB()

model.fit(X_train_Tfidf_df, y_train)

CategoricalNB()

In [13]:
y_pred = model.predict(X_test_Tfidf_df)
accuracy_score(y_test, y_pred)

NameError: name 'model' is not defined

In [93]:
sum = 0
for li in X:
    sum += len(li)
print(sum/len(X))
# mean len is 105
lengths = [len(sublist) for sublist in X]
sorted_lengths = sorted(lengths)
if len(sorted_lengths) % 2 == 0:
    # If the number of lengths is even, take the average of the middle two
    median_length = (sorted_lengths[len(sorted_lengths) // 2 - 1] + sorted_lengths[len(sorted_lengths) // 2]) / 2
else:
    # If the number of lengths is odd, take the middle value
    median_length = sorted_lengths[len(sorted_lengths) // 2]
print(median_length)
# median length is 40

105.05786873676782
40
Number of unique classes: 26


In [95]:
import tensorflow as tf
from collections import Counter
X = tech_training['combined']
y = tech_training['tag'].apply(lambda x: x[0])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_list = X.tolist()
y_list = y.tolist()

#y_val_counter = Counter(y_list)

#print("Counter of y_val:")
#print(y_val_counter)

# Split data into training and validation sets
x_train, x_tmp, y_train, y_tmp = train_test_split(X_list, y_list, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_tmp, y_tmp, test_size = 0.5, random_state=42)

# Tokenization and padding for training set
# max words is 13766
max_words = 5000  # Maximum number of words to consider
# max is 1097
maxlen = 60  # Maximum length of input sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)


sequences_train = tokenizer.texts_to_sequences(x_train)
x_train = tf.keras.preprocessing.sequence.pad_sequences(sequences_train, maxlen=maxlen)


# Tokenization and padding for validation set
sequences_val = tokenizer.texts_to_sequences(x_val)
x_val = tf.keras.preprocessing.sequence.pad_sequences(sequences_val, maxlen=maxlen)


sequences_test = tokenizer.texts_to_sequences(x_test)
x_test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test, maxlen=maxlen)

#y_train = np.array(y_train)
#y_test = np.array(y_test)
#y_val = np.array(y_val)

y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=26)
y_val_one_hot = tf.keras.utils.to_categorical(y_val, num_classes=26)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=26)

maxLen = 1097
minLen = 17
#vocab_size = len(set(word for seq in X for word in seq))
#max_sequence_length = max(len(seq) for seq in X)
#print(max_sequence_length)
#print(vocab_size)
print(x_train)


[[   0    0    0 ...    6  159  584]
 [  38  300  257 ...   73   38  300]
 [   0    0    0 ...  123  444    5]
 ...
 [1896  141 2784 ...  211    3   31]
 [   0    0    0 ...  343  108  190]
 [2016   14   70 ...   14  211    3]]


In [111]:

#from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

vocab_size = 13766
embedding_dim = 50
num_classes = 26


# Define the CNN model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 128, input_length=maxlen))
model.add(tf.keras.layers.Conv1D(128, 5, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(128, activation='relu'))  # Adding an additional dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))  # Adding an additional dense layer
model.add(tf.keras.layers.Dense(26, activation='softmax'))  # Output layer with softmax activation for 26 classes

# Compile the model
model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_one_hot, epochs=50, batch_size=100, validation_data=(x_val, y_val_one_hot))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test_one_hot)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')



Epoch 1/50


C:\Users\frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0342 - loss: 0.6701 - val_accuracy: 0.0188 - val_loss: 0.4877
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3012 - loss: 0.3836 - val_accuracy: 0.4836 - val_loss: 0.1954
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4170 - loss: 0.2040 - val_accuracy: 0.4836 - val_loss: 0.1426
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4189 - loss: 0.1427 - val_accuracy: 0.4836 - val_loss: 0.1149
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4491 - loss: 0.1162 - val_accuracy: 0.4930 - val_loss: 0.1081
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4676 - loss: 0.1100 - val_accuracy: 0.5305 - val_loss: 0.1032
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4991 - loss: 0.1043 - val_accuracy: 0.5305 - val_loss: 0.0998
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4635 - loss: 0.1014 - val_accuracy: 0.5305 - val_loss: 0.

In [97]:
predictions = model.predict(x_test)
print(predictions)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[2.04260042e-03 4.04395723e-06 2.16750358e-03 ... 6.88119617e-04
  7.29520200e-03 2.13790941e-03]
 [3.87937725e-02 4.20659262e-06 4.31872495e-02 ... 8.37649871e-03
  1.05160594e-01 1.94211807e-02]
 [1.91828050e-03 4.50491189e-06 1.96968298e-03 ... 6.81878359e-04
  7.48651568e-03 2.27166642e-03]
 ...
 [2.71669077e-03 3.37234542e-06 3.65792122e-03 ... 9.64716834e-04
  1.01820296e-02 2.87778839e-03]
 [7.19200075e-02 3.60305741e-04 1.74062103e-01 ... 4.72195409e-02
  4.29730415e-02 6.60036281e-02]
 [1.94396358e-03 3.24946518e-06 2.12599384e-03 ... 6.02450338e-04
  6.83868444e-03 2.07698881e-03]]
